<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional  # === Добавлено этап 9: для аннотации generate ===

class CausalSelfAttention(nn.Module):
    # === Добавлено на этапе 3: собственная реализация каузального внимания ===
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5
        self.qkv_proj = nn.Linear(embed_dim, 3 * embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.qkv_proj(x)  # [B, T, 3C]
        qkv = qkv.view(B, T, 3, self.num_heads, self.head_dim)
        q, k, v = qkv.unbind(dim=2)
        q = q.transpose(1, 2)  # [B, heads, T, head_dim]
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        attn_weights = (q @ k.transpose(-2, -1)) * self.scale  # [B, heads, T, T]
        mask = torch.tril(torch.ones(T, T, device=x.device)) == 0
        attn_weights = attn_weights.masked_fill(mask, float('-inf'))
        attn_probs = torch.softmax(attn_weights, dim=-1)
        attn_output = attn_probs @ v  # [B, heads, T, head_dim]
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, T, C)
        return self.out_proj(attn_output)
    # === Конец этапа 3 ===

class FeedForward(nn.Module):
    # === Добавлено на этапе 4: двухслойный FFN ===
    def __init__(self, embed_dim, hidden_dim=None):
        super().__init__()
        hidden_dim = hidden_dim or embed_dim * 4
        self.net = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, embed_dim),
        )

    def forward(self, x):
        return self.net(x)
    # === Конец этапа 4 ===

class TransformerBlock(nn.Module):
    # === Добавлено на этапе 5: разводим attention и mlp параллельно ===
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.attn = CausalSelfAttention(embed_dim, num_heads)
        self.mlp = FeedForward(embed_dim)
        self.ln_1 = nn.LayerNorm(embed_dim)
        self.ln_2 = nn.LayerNorm(embed_dim)
        self.threshold = 1.0  # можно регулировать чувствительность сверки

    def forward(self, x):
        # параллельные ветви: attention и mlp
        a = self.attn(self.ln_1(x))
        m = self.mlp(self.ln_2(x))

        # сверка: если сильно различаются, подавим результат
        diff = torch.norm(a - m, dim=-1, keepdim=True)  # [B, T, 1]
        mask = (diff < self.threshold).float()
        combined = mask * (a + m) / 2  # если различаются сильно, обнулим

        return x + combined
# === Конец модификации на этапе 8 ===

class MiniGPT(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, max_seq_len=128, num_layers=4):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)

        # === Добавлено на этапе 2: позиционные эмбеддинги ===
        self.position_embedding = nn.Embedding(max_seq_len, embed_dim)
        # === Конец этапа 2 ===

        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads) for _ in range(num_layers)
        ])

        # === Добавлено на этапе 7: финальный LayerNorm перед головой ===
        self.final_ln = nn.LayerNorm(embed_dim)
        # === Конец добавленного на этапе 7 кода ===

        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)

        self.max_seq_len = max_seq_len

    def forward(self, idx, targets=None):
        B, T = idx.size()
        positions = torch.arange(0, T, device=idx.device).unsqueeze(0)
        x = self.token_embedding(idx) + self.position_embedding(positions)

        for block in self.blocks:
            x = block(x)

        # === Добавлено на этапе 7: финальный LayerNorm перед головой ===
        x = self.final_ln(x)
        # === Конец добавленного на этапе 7 кода ===

        logits = self.lm_head(x)

        # === Добавлено на этапе 7: расчет loss при наличии targets ===
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        return logits, loss
        # === Конец добавленного на этапе 7 кода ===

    # === Добавлено этап 9: гибкая генерация (temperature, top_k) ===
    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature: float = 1.0, top_k: Optional[int] = None):
        """
        Генерирует `max_new_tokens`, начиная с `idx` (shape [B, T]).
        `temperature` < 1.0 делает вывод более «консервативным`, >1.0 — более «креативным`.
        `top_k` ограничивает выборку k наиболее вероятными токенами (top‑k sampling).
        """
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.max_seq_len:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature  # масштабируем температуру
            if top_k is not None:
                # оставим только top_k наивысших логитов, остальные -∞
                v, _ = torch.topk(logits, top_k)
                min_topk = v[:, -1].unsqueeze(-1)
                logits = torch.where(logits < min_topk, torch.full_like(logits, float('-inf')), logits)
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
        return idx
    # === Конец этапа 9 ===

# Пример использования
model = MiniGPT(vocab_size=1000, embed_dim=64, num_heads=4, num_layers=4)
tokens = torch.randint(0, 1000, (2, 5))
targets = torch.randint(0, 1000, (2, 5))
logits, loss = model(tokens, targets)
print("Logits shape:", logits.shape)
print("Loss:", loss.item())

# Пример гибкой генерации (temperature=0.8, top_k=40)
generated = model.generate(tokens[:, :2], max_new_tokens=5, temperature=0.8, top_k=40)
print("Generated tokens:", generated)


Logits shape: torch.Size([2, 5, 1000])
Loss: 7.0240020751953125
Generated tokens: tensor([[103, 278, 567, 553,   9,  17, 780],
        [765, 288, 255, 998, 577, 285, 125]])
